# Strava, Rouvy and Machine Learning

### How to predict 'moving time' on a route by Scikit-Learn
<center><img src="strava.png" alt="My Account from Strava web page" /><img src="rouvy.png" alt="My Account from Strava web page" /></center>

I've had a lot of fun riding my bike over the last few years. Unfortunately the Covid pandemic has greatly limited the opportunities for outdoor outings. Plus the winter is harsh in my area. So I subscribed to a nice app called (rouvy.com) and bought an indoor trainer to pedal at home. It was a fantastic experience that continues to this day. Early every day in the morning I can train by pedaling anywhere in the world, tackling the steepest and most legendary climbs. I connected Rouvy to my (free) Strava account and saved all the data related to the routes I have ridden. After 3 years the end result is that I have more than 500 routes (indoor and outdoor) saved in my Strava account. But how does Machine Learning come into play?
When I want to choose the next route to take, I often have only a rough idea of the "commute time" it will take. This is essential for scheduling my time. So I decided to use the data available in Strava to train a Machine Learning model and predict the "commute time" (also 'Moving time') given some parameters (distance, elevation gain, max speed, average speed...) available 'a priori' in the Rouvy profile of the route.
This notebook, the data and all the pictures are available under my github https://github.com/fabioantonini/strava-moving-time-regressor.
Let's dive.

## Retrieving data from Strava

The routes data can be exported by the Strava website from the 'My Account' page.

<center><img src="myaccount.png" alt="My Account from Strava web page" /></center>

Search for 'Download or Delete Your Account'. Click on the 'Get Started'.

<img src="export.png" alt="My Account from Strava web page" />

Click on the 'Request Your Archive' button. As explained, an email will be sent to you with the link to download the zip file containing the data of your Strava activities. Prepare to wait for a while. Strava takes its time to arrange the archive. So you might receive the email after some hours.
Anyway in the end you will receive the email and download the zip file (export_31174850.zip for instance).
Let's take a look at its content.

<img src="zipfile.png" alt="My Account from Strava web page" />

For our purposes only the 'activities.csv' file is required. From the size we can realize that it contains a lot of data. My own 'activities.csv' file has been added to the repo and it will be processed. Let's import it.

In [ ]:
import numpy
import pandas
import matplotlib.pyplot as plt
import scipy.stats
from sklearn.linear_model import LinearRegression # Regression Model
from sklearn.model_selection import train_test_split # to split train and test sets
plt.style.use("bmh")
%config InlineBackend.figure_formats=["png"]

Let's assume to have already downloaded the activities.csv file from our Strava profile.

In [ ]:
activities = pandas.read_csv("activities.csv")
print("dataset type is:", type(activities), "length:", len(activities), "shape:", activities.shape)

The dataset is really rich. It includes 86 columns for each route. 
Not all the columns contain usable data because I don't have a full Strava subscription, but only a free account. Let's take a look more in depth to undestand which activities are really useful to out purpose.

In [ ]:
print("columns: ", len(list(activities.columns)))

In [ ]:
activities.head()

In [ ]:
activities.info()

Of course not all the columns are helpful. So let's extract only the columns really useful to train a model.
The data appear to be a bit sparsed. Some columns are not valorized at all (because of my free account). Others columns are many 'null'' values. We need to indetify only the features statistically helpful.

## Data cleaning

In the next section data will be cleaned and filtered to get only routes done by bike (Outdoor and Indoor).

### Getting only activities done by bike

We defintely need to get only the activities done by bike. They are labeled as 'Ride' and 'Virtual Ride' in the Strava exported dataset. So we will drop the activities tagged as 'Walk' and 'Run'.

In [ ]:
activities=activities.loc[activities['Activity Type'].isin(['Ride', 'Virtual Ride'])]

In [ ]:
activities.head(5)

In [ ]:
# as an alternative you can remove the 'Walk' and 'Run' activities
# activities = activities.drop(activities[activities['Activity Type'] == 'Walk'].index)
# activities = activities.drop(activities[activities['Activity Type'] == 'Run'].index)

### Removing short routes

Rouvy usually provides a 'Warm up' and 'Cool down' short routes before and after respectively the selected route. So also this short routes have been recorded under Strava. They are not useful for our purposes. So let's remove all the routes whose 'Moving Time' is less than 3 minutes (180 secs).

In [ ]:
activities = activities.drop(activities[activities['Moving Time'] < 180].index)

In [ ]:
activities.head()

### Handling fake data

Inspecting the original dataframe we can realize that there are some bad data. For instance it's hard to believe that the 'Max Grade' is 50%. If the min 'Distance' is 0 Km, the route is a 'fake' or the data are corrupted. So these routes that can be removed.
Let's clean this data by setting a threshold of 25% for the 'Max Grade', and 3 Km's for 'Distance' respectively.

In [ ]:
activities = activities.drop(activities[activities['Max Grade'] > 25].index)

In [ ]:
activities = activities.drop(activities[activities['Distance'] < 3].index)

In [ ]:
activities.describe()

In [ ]:
activities.head()

Now the data looks better.

### Selecting features and labels

After a fast analysis of the available features, only the following features will be used to train the model:
- Distance
- Elevation gain
- Max Grade
- Average Grade

The target label is the 'Moving Time'.

These informations can be retrieved by Rouvy, for every available route.
Please note that in the picture the Elevation Gain is mapped to the 'Ascended' item.

<center><img src="rouvy-route.png" alt="Rouvy data for a route" /></center>

In [ ]:
activities = activities[["Distance", "Elevation Gain", "Max Grade", "Average Grade", "Moving Time"]]
activities.describe()

We can notice that not all the feature have the same number of not null values.

### Handling 'Null' values

We can check the 'null' values and remove the rows with at least one 'null' value.

In [ ]:
if activities.isnull().values.any():
    activities=activities.dropna()
activities.describe()

We have 530 records (rows) that can be used to train our model.

### Save the cleaned data

In [ ]:
activities.to_csv('cleaned_activities.csv')

In [ ]:
activities.head()

Let's take a look at the dataframe after the last processing.

## Visualization

We can obtain a first impression of the dependency between variables by examining a multidimensional scatterplot.

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(activities, diagonal="kde", figsize=(12,10));

As expected we can see a linear relationship between the Moving Time and the Distance.

In [ ]:
activities.plot(kind="scatter", x='Distance', y='Moving Time', grid=True)

there is an approximately linear relationship between Elevation Gain and the Distance: more Kms more the overall gain in altitude

In [ ]:
activities.plot(kind="scatter", x='Distance', y='Elevation Gain', grid=True)

We can also generate a 3D plot of the observations, which can sometimes help to interpret the data more easily. Here we plot 'Moving Time' as a function of 'Distance' and 'Elevation Gain'.

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(projection="3d")
ax.scatter(activities["Distance"], activities["Elevation Gain"], activities["Moving Time"])
ax.set_xlabel("Distance")
ax.set_ylabel("Elevation Gain")
ax.set_zlabel("Moving Time")
ax.set_facecolor("white")

In [ ]:
%matplotlib inline
activities.hist(bins=50, figsize=(20,15))
plt.show()

## Looking for correlation

You can easily compute the standard correlation coefficient (also called Pearson's r) between every pair of attributes using the 'corr()' method.

In [ ]:
corr_matrix= activities.corr()

In [ ]:
corr_matrix["Moving Time"].sort_values(ascending=False)

The Moving Time is strongly correlated to the 'Distance' and also to the 'Elevation Gain'. This is asbolutely expected.

## Getting the 'labels' and 'features'

In [ ]:
labels=activities.pop("Moving Time")

In [ ]:
print("label shape:", labels.shape, "and type:", type(labels))

In [ ]:
print("features shape:", activities.shape, "and type:", type(activities))

## Split training and test set

In [ ]:
X = activities.values  # values converts it into a numpy array
Y = labels.values

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

## Linear Regression Model

We will created a fitted linear model using the formula API of the scikit-learn library.

In [ ]:
linear_model = LinearRegression()
linear_model.fit(X_train, Y_train) 

### View Parameters 
The $\mathbf{w}$ and $\mathbf{b}$ parameters are referred to as 'coefficients' and 'intercept' in scikit-learn. In other term the model function can be written as $f_{w,b}(\vec{x})$

In [ ]:
b = linear_model.intercept_
w = linear_model.coef_
print(f"w = {w:}, b = {b:0.2f}")

Let's give it a try by using some routes from the Test set.

In [ ]:
some_data=X_test[5:10,:]
some_labels=Y_test[5:10]

In [ ]:
some_labels_predicted = linear_model.predict(some_data)

In [ ]:
print("Error (mins):", (some_labels_predicted - some_labels)/60)

### Mean squared error

Let’s measure the regression model’s RMSE on the whole training set using Scikit-Learn’s mean_squared_error function.

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
lin_mse = mean_squared_error(some_labels, some_labels_predicted)
lin_rmse = numpy.sqrt(lin_mse)
lin_rmse/60

It's a little bit high. Something doesn't work as expected. The Linear model assumption seems to be not right.

## Calculate accuracy

You can calculate this accuracy of this model by calling the `score` function on the whole Training set.

In [ ]:
print("Accuracy on training set:", linear_model.score(X_train, Y_train))

The model is doing poor on the Training set. It's too simple and it can match the training data at the best. Let's try another model.

In [ ]:
# print("Accuracy on test set:", linear_model.score(X_test, Y_test))